In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from lazypredict.Supervised import LazyClassifier
import pickle

# Step1: Read input
data = pd.read_csv("../datasets/diabetes.csv")

profile = ProfileReport(data, title="Diabetes Report", explorative=True)
profile.to_file("diabetes.html")

# Step2: Divide the dataset
target = "Outcome"
x = data.drop(target, axis=1)  # axis=0: get row
y = data[target]

# Chia du lieu thanh train, test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2201)  # random_state: Đảm bảo mỗi lần chia train và test không đổi
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=2201)

# Step3: Data preprocessing
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Bộ test không bao giờ được tham gia vào quá trình tính Mean, StandardDiviation
# scaler.fit(x_train)
# scaler.transform(x_train) # dùng với bộ test
# scaler.fit_transform()  # dùng với bộ train


# Step4: Select model and training

# SVC for classification
# model = SVC(probability=True)
# model.fit(x_train, y_train)
# y_predict = model.predict_proba(x_test)
# print(y_predict)

# print("Accuracy: {}".format(accuracy_score(y_test, y_predict)))
# print("Precision: {}".format(precision_score(y_test, y_predict)))
# print("Recall: {}".format(recall_score(y_test, y_predict)))
# print("F1: {}".format(f1_score(y_test, y_predict)))
# print(classification_report(y_test, y_predict))

# threshold = 0.3
# for score in y_predict:
#     if score[1] > threshold:
#         print("Class 1")
#     else:
#         print("Class 0")

# model = LogisticRegression()
# model = RandomForestClassifier(n_estimators=100, criterion="gini", random_state=2201)

params = {
    "n_estimators": [50, 100, 200], # so luong cay
    "criterion": ["gini", "entropy", "log_loss"],  # tieu chi
    "max_depth": [None, 2, 5]  # do sau cua cay
}

model = GridSearchCV(
    estimator=RandomForestClassifier(random_state=2201),
    param_grid=params,
    scoring="recall",
    cv=6, # k - fault
    verbose=1,  #in ra it hay nhieu
    n_jobs=8  #so luong chay
)
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
print(model.best_params_)
print(model.best_score_)
print(classification_report(y_test, y_predict))


# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)  # train thu nghiem cho nhieu mo hinh, cac tham so deu dung mac dinh
# models,predictions = clf.fit(x_train, x_test, y_train, y_test)

with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

# print(x_train.shape, y_train.shape)
# print(x_test.shape, y_test.shape)

# print(data)

Fitting 6 folds for each of 27 candidates, totalling 162 fits
{'criterion': 'gini', 'max_depth': None, 'n_estimators': 100}
0.5705026455026455
Accuracy: 0.7402597402597403
Precision: 0.64
Recall: 0.5925925925925926
F1: 0.6153846153846154
